---
# Drzewa decyzyjne #
---

## 1. Pobranie i analiza danych uczących ##
Będziemy dziś pracować na [zbiorze danych](http://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+%28diagnostic%29) uzyskanych z biopsji (aspiracyjnej cienkoigłowej) piersi. Zadaniem naszym jest wytrenowanie klasyfikatora, który po cechach wyników biopsji będzie klasyfikował guz jako łagodny lub złośliwy.

In [1]:
from sklearn.datasets import load_breast_cancer

# Gdy znacie zestaw danych i chcecie od razu przejść do treningu, opcja 'return_X_y = True' jest dla Was!
#X, y = load_breast_cancer(return_X_y=True)

# Ale to nasz pierwszy raz z tym zestawem danych, więc sprawdźmy co jest w środku. Wykorzystajcie analizę danych zrobioną dla irysów w notebooku z naiwnym klasyfikatorem Bayesa.
cancer = load_breast_cancer()

# Sprawdźcie opis zestawu danych, a także nazwy klas i nazwy cech
# Sprawdźcie ile jest przykładów z obu klas
# Zróbcie violinploty dla wszystkich cech LUB przyjrzyjcie się zakresom liczbowym cech. 
# Co o nich myślicie? Czy sprawią jakiś problem? Czy powinniśmy coś z nimi zrobić? PODPOWIEDŹ: TAK ;D
# Jeśli nie możecie wymyślić gdzie leży pułapka, to dajcie mi znać, nie idźcie dalej, wymyślimy to razem :)

In [ ]:
# Zróbcie wykres korelacji między cechami (tym razem prawdziwej korelacji ;) nie jak w irysach!
# X zawiera Wasze dane uczące
X, y = load_breast_cancer(return_X_y=True)

plt.figure()
f, ax = plt.subplots(figsize=(14,14))
corr_plot = sns.heatmap(np.corrcoef(X, rowvar=False), annot=False, linewidths=.5, fmt='.1f', ax=ax)

# Czego się dowiedzieliście z tego wykresu?

In [ ]:
from sklearn.model_selection import train_test_split

# Podziel dane na treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Tutaj unikamy pułapki z komórki nr 1 :)

## 2. Importujemy klasyfikator w postaci drzewa decyzyjnego ##
W bibliotece scikit-learn drzewa decyzyjne implementowane są przez klasę [DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier). Szczegóły implementacji opisane są [tutaj](https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use).

Najprostszy przykład:

In [ ]:
from sklearn import tree
X = [[0, 0], [1, 1]]
Y = [0, 1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

Po dopasowaniu można przewidywać przynależność nowych przykładów:

In [ ]:
clf.predict([[2., 2.]])

array([1])

Albo estymować prawdopodobieństwo przynależności do klas:

In [ ]:
clf.predict_proba([[2., 2.]])

array([[0., 1.]])

## 3. Klasyfikacja guzów i wizualizacja drzewa ##

In [ ]:
# Deklarujemy klasyfikator

# Fitujemy do danych treningowych


Po wytrenowaniu można zilustrować wynik za pomocą narzędzia Graphiz (wymaga to zainstalowania w systemie tego narzędzia), oraz doinstalowania do pythona biblioteki pydot:

In [ ]:
# Nazwałam tu klasyfikator 'clf'. Jeśli zadeklarowaliście go pod inną nazwą, to:
# clf = Wasza_Nazwa

from six import StringIO
import pydot 
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph[0].write_pdf("iris.pdf")

Można też podejrzeć wyniki w pythonie:

In [ ]:
from IPython.display import Image  
dot_data = StringIO()  
tree.export_graphviz(clf, out_file=dot_data, feature_names=cancer.feature_names,  
                         class_names=cancer.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True) 
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph[0].create_png())

Czym jest "Gini", "samples", "value" oraz "class"? Co oznaczają kolory?

In [ ]:
# Alternatywą jest użycie wbudowanej metody 'plot_tree', ale wydaje mi się mniej przyjazna:
tree.plot_tree(clf, feature_names=cancer.feature_names)

Oczywiście oceniamy jakość tej klasyfikacji:

In [ ]:
# Raport jakości klasyfikacji + Wasze ulubione miary jakości.

Co o niej myślicie? A sprawdźcie na wszelki wypadek klasyfikację danych treningowych?

In [ ]:
# Raport klasyfikacji w procesie treningowym

Co to oznacza? :)

## 4. Regularyzacja drzewa przez przycinanie##

A teraz crème de la crème uczenia maszynowego, czyli regularyzacja. Pamiętacie na czym ogólnie polega?

W przypadku drzew decyzyjnych polega przede wszystkim na:
- przycinaniu gałęzi i liści (czyli zmniejszaniu liczby węzłów) przez kontrolę parametru min_samples_leaf: mianowicie daje dolny limit na jak drobne gałązki może drzewo tworzyć. Jeśli kolejna gałązka miałaby rozpatrywać tylko liczbę przykładów < min_sample_leaf, to nie tworzy się. Domyślnie, min_sample_leaf = ?
- ograniczaniu głębokości drzewa, czyli jak wiele rozgałęzień może się stworzyć. Domyślnie max_depth = None. Co to znaczy? Patrz [tu](https://scikit-learn.org/0.22/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier).

Macie dwa wyjścia. Po pierwsze, poszukać optymalnych hiperparametrów ręcznie, jak ostatnio robiliście. Po drugie, skorzystać (z rozwagą!) z gotowej metody cost_complexity_pruning_path, w której bawicie się jednym parametrem 'cost complexity parameter', czyli ccp_alpha. Tutaj macie [tutorial](https://scikit-learn.org/0.22/auto_examples/tree/plot_cost_complexity_pruning.html). Jeśli będziecie korzystać z tej funkcji, to proszę o bogate komentarze, co się po drodze dzieje :)

Ostatecznym wynikiem ma być uzyskanie optymalnego drzewa oraz jego wizualizacja wraz z raportem klasyfikacji i macierzą pomyłek.

Powodzenia! :)